# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [212]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import ipywidgets as widgets
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [213]:
file = '../Resources/weather_call_data.csv'
cities_df = pd.read_csv(file)
del cities_df['Unnamed: 0']
cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
0,Rikitea,97,PF,1580703915,79,-23.12,-134.97,26.10,6.58
1,Tuktoyaktuk,5,CA,1580703915,73,69.45,-133.04,-38.00,3.10
2,Matamoros,75,MX,1580703913,48,25.53,-103.25,18.00,2.82
3,Severo-Kuril'sk,55,RU,1580703915,83,50.68,156.12,-7.50,4.43
4,Vila Franca do Campo,20,PT,1580703916,87,37.72,-25.43,10.00,2.60
...,...,...,...,...,...,...,...,...,...
544,Husavik,100,IS,1580704094,84,66.04,-17.34,-16.00,0.50
545,Naron,0,ES,1580704094,82,43.52,-8.15,18.89,3.10
546,Collie,0,AU,1580704094,18,-33.37,116.15,32.22,4.28
547,Kuytun,0,RU,1580704094,94,54.34,101.50,-29.00,1.21


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [214]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)
cities_df['Lat'], cities_df['Lng']

(0     -23.12
 1      69.45
 2      25.53
 3      50.68
 4      37.72
        ...  
 544    66.04
 545    43.52
 546   -33.37
 547    54.34
 548   -23.13
 Name: Lat, Length: 549, dtype: float64, 0     -134.97
 1     -133.04
 2     -103.25
 3      156.12
 4      -25.43
         ...  
 544    -17.34
 545     -8.15
 546    116.15
 547    101.50
 548    150.73
 Name: Lng, Length: 549, dtype: float64)

In [215]:
#Create coordinate listing
coordinates = cities_df[["Lat", "Lng"]].astype(float)

#create variable for humidity
humidities = cities_df['Humidity'].astype(float)

In [217]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidities, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)
#add heat layer
fig.add_layer(heat_layer)

#display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [218]:
#display maximum values for all numeric fields to find achievable conditions
cities_df.max()

City            Zyryanka
Cloudiness           100
Date          1580704094
Humidity             100
Lat                78.22
Lng               179.32
Max_Temp              38
Wind_Speed            20
dtype: object

In [219]:
#use .loc filtering to narrow cities to fit ideal weather condition one (humidity <50)
cities_df_rest = cities_df.loc[cities_df['Humidity'] < 50]

In [220]:
#use .loc filtering to narrow cities to fit ideal weather condition tw0 (windspeed <15)
cities_df_rest = cities_df_rest.loc[cities_df['Wind_Speed']<15]

In [221]:
#use .loc filtering to narrow cities to fit ideal weather condition three (cloudiness > 76)
cities_df_rest = cities_df_rest.loc[cities_df['Cloudiness']>76]

In [222]:
#drop any rows with null values
cities_df_rest.dropna()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
41,Esperance,93,AU,1580703627,43,-33.87,121.90,23.00,3.60
73,Paamiut,83,GL,1580703930,38,61.99,-49.67,-6.69,3.98
128,Puerto Ayora,85,EC,1580703941,11,-0.74,-90.35,25.56,1.79
141,Seddon,90,AU,1580703944,48,-37.81,144.89,18.33,9.30
244,Kyzyl-Suu,87,KG,1580703966,45,42.34,78.00,-3.88,2.51
398,Frontera,100,MX,1580704063,32,26.93,-101.45,17.35,1.05
441,Ouesso,81,CG,1580704072,40,1.61,16.05,21.40,1.31
491,Aswan,83,EG,1580704082,35,24.09,32.91,11.23,4.83


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [223]:
#store into variable named hotel_df
hotel_df = cities_df_rest

hotel_df[['Lat','Lng']]

,Lat,Lng
41,-33.87,121.90
73,61.99,-49.67
128,-0.74,-90.35
141,-37.81,144.89
244,42.34,78.00
398,26.93,-101.45
441,1.61,16.05
491,24.09,32.91


In [224]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Create coordinate listing column for ease of looping
target_coordinates = []
hotel_df['coordinates'] = hotel_df['Lat'].apply(str) + str(', ') +hotel_df['Lng'].apply(str)
hotel_df['coordinates']

41      -33.87, 121.9
73      61.99, -49.67
128     -0.74, -90.35
141    -37.81, 144.89
244       42.34, 78.0
398    26.93, -101.45
441       1.61, 16.05
491      24.09, 32.91
Name: coordinates, dtype: object

In [225]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#hotel variable
hotels = []
coordinates = hotel_df['coordinates'].apply(str)

# geocoordinates 
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary and use a for loop to iterate through the loop

for x in coordinates:
    params = {
    "location": str(f"{x}"),
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }
    
    # response construction using base url and params dictionary
    response = requests.get(base_url, params=params)
    places_data = response.json()
    
    # Visualize the loop run
    print(f'The closest hotel to location {x}: {places_data["results"][0]["name"]}')
    
    #append the hotel name to the hotels variable for inclusion in the dataframe
    hotels.append(places_data["results"][0]["name"])


The closest hotel to location -33.87, 121.9: Esperance Island View Apartments
The closest hotel to location 61.99, -49.67: Yeet Hotel
The closest hotel to location -0.74, -90.35: Hotel Solymar Galapagos
The closest hotel to location -37.81, 144.89: Four Points by Sheraton Melbourne Docklands
The closest hotel to location 42.34, 78.0: Gostinitsa Nur-Ordo
The closest hotel to location 26.93, -101.45: Holiday Inn Monclova
The closest hotel to location 1.61, 16.05: Hotel Mindongo Ouesso
The closest hotel to location 24.09, 32.91: Hotel Sofitel Legend Old Cataract Aswan


In [226]:
#Add hotel name column and populate with results of the loop above
hotel_df.insert(9,'Hotel Name',hotels)

#display updated df
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed,Hotel Name,coordinates
41,Esperance,93,AU,1580703627,43,-33.87,121.90,23.00,3.60,Esperance Island View Apartments,"-33.87, 121.9"
73,Paamiut,83,GL,1580703930,38,61.99,-49.67,-6.69,3.98,Yeet Hotel,"61.99, -49.67"
128,Puerto Ayora,85,EC,1580703941,11,-0.74,-90.35,25.56,1.79,Hotel Solymar Galapagos,"-0.74, -90.35"
141,Seddon,90,AU,1580703944,48,-37.81,144.89,18.33,9.30,Four Points by Sheraton Melbourne Docklands,"-37.81, 144.89"
244,Kyzyl-Suu,87,KG,1580703966,45,42.34,78.00,-3.88,2.51,Gostinitsa Nur-Ordo,"42.34, 78.0"
398,Frontera,100,MX,1580704063,32,26.93,-101.45,17.35,1.05,Holiday Inn Monclova,"26.93, -101.45"
441,Ouesso,81,CG,1580704072,40,1.61,16.05,21.40,1.31,Hotel Mindongo Ouesso,"1.61, 16.05"
491,Aswan,83,EG,1580704082,35,24.09,32.91,11.23,4.83,Hotel Sofitel Legend Old Cataract Aswan,"24.09, 32.91"


In [227]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in cities_df_rest.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [232]:
# Add marker layer ontop of heat map with infoboxes
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))